In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import re
import pickle

#### Positional Encodings
Method to get the angles
d_model is the dimemsion of the embeddings
This method returns pos / (10000)^(2*i/d_model)


In [5]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2) / np.float32(d_model)))
    return pos * angle_rates